# Задача

Анализ лауреатов нобелевской премии.

# Подготовка окружения

Задание выполнено в Google Colab, и в этой секции я подтягиваю код напрямую из репозитория.
Проверяющему проект это скорее всего не нужно, можно пропустить эту секцию .

Первичное клонирование репозитория:

In [2]:
!git clone -b my-final-task --single-branch https://github.com/shasha-sh/aaa-final-task.git
%cd aaa-final-task

Cloning into 'aaa-final-task'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 14 (delta 4), reused 9 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (14/14), done.
Resolving deltas: 100% (4/4), done.
/content/aaa-final-task


Обновление кода с github:

In [ ]:
%cd /content/aaa-final-task
!git fetch origin my-final-task
!git checkout my-final-task
!git pull --rebase origin my-final-task

/content/aaa-final-task
From https://github.com/shasha-sh/aaa-final-task
 * branch            my-final-task -> FETCH_HEAD
Already on 'my-final-task'
Your branch is up to date with 'origin/my-final-task'.
From https://github.com/shasha-sh/aaa-final-task
 * branch            my-final-task -> FETCH_HEAD
Already up to date.


Сброс кэша импортов:

In [ ]:
%cd /content/aaa-final-task
import importlib, json_dict_processing, prizes_configs, laureates_configs
importlib.reload(json_dict_processing)
importlib.reload(prizes_configs)
importlib.reload(laureates_configs)

/content/aaa-final-task


<module 'laureates_configs' from '/content/aaa-final-task/laureates_configs.py'>

# Загрузка данных

В этом разделе я подготавливаю данные для анализа :
  1) Получаю данные о лауреатах Нобелевского фонда.
  2) Привожу данные к единому плоскому формату, чтобы дальше с ними было удобно работать.

Необходимые данные находятся публичном API Нобелевского фонда, откуда ответ приходит в формате JSON-словаря. Поэтому данные не требуют сложной предобработки - достаточно проверить базовую корректность структуры ответа.

In [3]:
import requests
URL_LAUREATES = f'https://api.nobelprize.org/2.1/laureates?limit=1200'

laureates_response = requests.get(URL_LAUREATES)
laureates = laureates_response.json()

assert isinstance(laureates, dict)
assert "laureates" in laureates

laureates_list = laureates["laureates"]

assert isinstance(laureates_list, list)
assert len(laureates_list) > 0
assert isinstance(laureates_list[0], dict)
assert all([("knownName" in x) or ("orgName" in x) for x in laureates_list])

Номинантами на премию могут быть люди или организации, и в данных они записаны по-разному. Различаем их по ключам: у людей есть `knownName`, у организаций - `orgName`.  Код проходится по каждому объекту, вспомогательная функция определяет тип записи по этим ключам и приводит её к плоскому, однообразному формату.

**Если какого-то поля в исходных данных нет, в результирующем словаре будет None.**

In [4]:
from laureates_configs import process_orgs, process_persons


def get_laureate_data(laureate: dict) -> dict | None:
    """
    Normalizes a laureate record and labels its type.

    :param laureate: Input laureate dict (from the API).
    :return: Normalized dict with 'laureate_type', or None if undetermined.
    """
    if "knownName" in laureate:
        d = process_persons(laureate)
        d["laureate_type"] = "person"
        return d
    if "orgName" in laureate:
        d = process_orgs(laureate)
        d["laureate_type"] = "org"
        return d
    return None


laureates_data = []

for laureate in laureates['laureates']:
    laureates_data.append(get_laureate_data(laureate))

assert isinstance(laureates_data, list)
assert all((laureate is None) or isinstance(laureate, dict)
                                        for laureate in laureates_data)

 # Формат данных


Перед анализом зафиксируем формат нормализованных данных. Вся информация извлечена на английском языке. В скобках указаны соответствующие пути к полям в исходном JSON.


**Общие поля нормализованных записей**

- `id` — уникальный идентификатор лауреата *(из `id`)*  
- `name` — имя человека или название организации *(из `knownName.en` / `orgName.en`)*  
- `country_now` — современное название страны, где родился человек или где основана организация *(из `birth.place.countryNow.en` / `founded.place.countryNow.en`)*  
- `laureate_type` — тип лауреата: `"person"` или `"org"` *(добавлено на этапе нормализации)*  
- `prizes_relevant` — список призов этого лауреата (см. структуру ниже) *(из `nobelPrizes`, обрабатывается процессором призов)*


**Только для людей** (`laureate_type = "person"`)
- `gender` — пол *(из `gender`)*  
- `birth_year` — год рождения *(год из строки `birth.date`)*  
- `country_birth` — страна рождения *(из `birth.place.country.en`)*


**Только для организаций** (`laureate_type = "org"`)
- `founded_year` — год основания организации *(год из строки `founded.date`)*  
- `country_founded` — страна основания *(из `founded.place.country.en`)*


**Структура элемента в `prizes_relevant`** (каждый элемент — один приз)
- `award_year` — год вручения *(из `awardYear`)*  
- `category_en` — категория приза, напр. _Physics_, _Peace_, _Economic Sciences_ *(из `category.en`)*  
- `prize_amount` — сумма приза на момент вручения *(из `prizeAmount`)*  
- `prize_amount_adjusted` — сумма с поправкой на инфляцию *(из `prizeAmountAdjusted`)*  
- `prize_status` — статус приза *(из `prizeStatus`)*


# Анализ данных


## Вспомогательный функционал для анализа

При анализе понадобится считать несколько независимых метрик. `count_with_conditions` делает это за один проход по данным, используя заданный список условий и список соответствующих названий результатов.

*  Принимает список нормализованных записей (records), список предикатов (conditions) и такой же по длине список названий результатов (labels).

*  Для каждой записи по очереди применяет каждый предикат; если предикат вернул True, увеличивает соответствующий счётчик.

*  Ошибки внутри предиката для конкретной записи игнорируются.

In [17]:
from typing import Callable

def count_with_conditions(
    records: list[dict[str, object]],
    conditions: list[Callable[[dict[str, object]], bool]],
    labels: list[str]
) -> dict[str, int]:
    """
    Counts how many records satisfy each condition.

    :param records: Normalized laureate dicts.
    :param conditions: List of predicates: record -> bool.
    :param labels: Names for results for each condition.
    :return: Dict {label: count}.
    """
    if len(conditions) != len(labels):
        raise ValueError("conditions and labels must have the same length")

    result: dict[str, int] = {name: 0 for name in labels}

    for rec in records:
        for cond, name in zip(conditions, labels):
            try:
                if cond(rec):
                    result[name] += 1
            except Exception:
                pass

    return result

## EDA лауреатов

На прошлом этапе мы работали только с записями, которые относятся к людям и организациям, исследую два конкретных формата словаря. Проверим не упустили ли мы еще что-то.

In [24]:
total_count = count_with_conditions(
    laureates_data,
    conditions=[lambda d: True, lambda d: (d is not None)],
    labels=["total", "normalized"]
)

print(f"Total record count (via counter): {total_count["total"]}")
print(f"Normalized record count (via counter): {total_count["normalized"]}")


Total record count (via counter): 1018
Normalized record count (via counter): 1018


Таким образом количества записей относящихся либо к людям, либо к организациям равно искомому количеству записей, то есть мы нормализовали всех существующих лауреатов. Сохраним это значение.

In [25]:
n = total_count["total"]

Сразу посчитаем количество лауреатов-людей и лауреатов-организаций.

In [26]:
type_counts = count_with_conditions(
    laureates_data,
    conditions=[lambda d: d["laureate_type"] == "person",
                lambda d: d["laureate_type"] == "org"],
    labels=["person_count", "org_count"]
)

print(f"Total record count (via counter): {type_counts["person_count"]}")
print(f"Normalized record count (via counter): {type_counts["org_count"]}")

Total record count (via counter): 990
Normalized record count (via counter): 28


Заметим, что количество лауреатов-людей кратно больше.

Проверим насколько полная информация у нас о пользователях.

In [35]:
n = len(laureates_data)

FIELDS = [
    "id", "name", "laureate_type", "country_now",
    "gender", "birth_year", "country_birth",
    "founded_year", "country_founded",
    "prizes_relevant",
]

missing_counts = count_with_conditions(
    laureates_data,
    conditions=[(lambda f:
                  (lambda d: (f not in d) or (d.get(f) is None))
                )(f) for f in FIELDS],
    labels=FIELDS,
)

missing_ratios = {k: v / n for k, v in missing_counts.items()}

print("Missing (counts):")
for k in FIELDS:
    print(f"  {k}: {missing_counts[k]}")

print("Missing (ratios):")
for k in FIELDS:
    print(f"  {k}: {missing_ratios[k]:.3f}")


Missing (counts):
  id: 0
  name: 0
  laureate_type: 0
  country_now: 8
  gender: 28
  birth_year: 28
  country_birth: 32
  founded_year: 991
  country_founded: 994
  prizes_relevant: 0
Missing (ratios):
  id: 0.000
  name: 0.000
  laureate_type: 0.000
  country_now: 0.008
  gender: 0.028
  birth_year: 0.028
  country_birth: 0.031
  founded_year: 0.973
  country_founded: 0.976
  prizes_relevant: 0.000
